<a href="https://colab.research.google.com/github/AlmahdiPen/2025_PBO_TI-1A/blob/main/aplikasi_PBO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok pandas -q

In [ ]:
%%writefile model.py

from dataclasses import dataclass, field
from typing import List

@dataclass
class Bahan:
    """Mendefinisikan satu bahan dengan jumlah dan satuan."""
    nama: str
    jumlah: float
    satuan: str

    def __repr__(self):
        return f"{self.jumlah} {self.satuan} {self.nama}"

@dataclass
class Resep:
    """Mendefinisikan satu resep, yang terdiri dari banyak bahan."""
    nama: str
    bahan: List[Bahan] = field(default_factory=list)
    instruksi: str = ""

    def __repr__(self):
        return f"Resep({self.nama})"

Overwriting model.py


In [ ]:
%%writefile manajer_data.py

import json
from model import Resep, Bahan
from typing import List, Dict

class ManajerData:
    """Mengelola proses simpan, muat, tambah, hapus, dan update resep."""
    def __init__(self, filepath="resep_db.json"):
        self.filepath = filepath

    def _resep_ke_dict(self, resep: Resep) -> Dict:
        return {
            "nama": resep.nama,
            "bahan": [{"nama": b.nama, "jumlah": b.jumlah, "satuan": b.satuan} for b in resep.bahan],
            "instruksi": resep.instruksi
        }

    def muat_resep(self) -> List[Resep]:
        try:
            with open(self.filepath, 'r') as f:
                data = json.load(f)
                return [Resep(
                    nama=rd['nama'],
                    bahan=[Bahan(**b) for b in rd.get('bahan', [])],
                    instruksi=rd.get('instruksi', '')
                ) for rd in data]
        except (FileNotFoundError, json.JSONDecodeError):
            return []

    def simpan_resep(self, daftar_resep: List[Resep]):
        with open(self.filepath, 'w') as f:
            json.dump([self._resep_ke_dict(r) for r in daftar_resep], f, indent=4)

    def tambah_resep(self, resep: Resep):
        daftar_resep = self.muat_resep()
        if any(r.nama.lower() == resep.nama.lower() for r in daftar_resep):
            raise ValueError(f"Resep dengan nama '{resep.nama}' sudah ada.")
        daftar_resep.append(resep)
        self.simpan_resep(daftar_resep)

    def hapus_resep(self, nama_resep: str):
        daftar_resep = self.muat_resep()
        resep_setelah_hapus = [r for r in daftar_resep if r.nama.lower() != nama_resep.lower()]
        if len(resep_setelah_hapus) == len(daftar_resep):
            raise ValueError(f"Resep '{nama_resep}' tidak ditemukan.")
        self.simpan_resep(resep_setelah_hapus)

    def update_resep(self, nama_resep_lama: str, resep_baru: Resep):
        daftar_resep = self.muat_resep()
        index_to_update = -1
        for i, r in enumerate(daftar_resep):
            if r.nama.lower() == nama_resep_lama.lower():
                index_to_update = i
                break

        if index_to_update == -1:
            raise ValueError(f"Resep '{nama_resep_lama}' tidak ditemukan untuk diupdate.")

        if nama_resep_lama.lower() != resep_baru.nama.lower():
             if any(r.nama.lower() == resep_baru.nama.lower() for r in daftar_resep):
                raise ValueError(f"Nama resep baru '{resep_baru.nama}' sudah digunakan oleh resep lain.")

        daftar_resep[index_to_update] = resep_baru
        self.simpan_resep(daftar_resep)

Overwriting manajer_data.py


In [ ]:
%%writefile perencana.py

from collections import defaultdict
from typing import List, Dict
from model import Resep

class PerencanaMenu:
    """Berisi logika bisnis untuk menghasilkan daftar belanja."""
    def __init__(self, rencana_menu: Dict, semua_resep: List[Resep]):
        self.rencana_menu = rencana_menu
        self.semua_resep_dict = {resep.nama: resep for resep in semua_resep}

    def buat_daftar_belanja(self) -> Dict:
        """
        Mengagregasi semua bahan dari rencana menu menjadi satu daftar belanja.
        Keluaran: Dictionary {(nama_bahan, satuan): total_jumlah}
        """
        daftar_belanja = defaultdict(float)

        for waktu_makan, daftar_resep_harian in self.rencana_menu.items():
            for nama_resep in daftar_resep_harian:
                if nama_resep and nama_resep != "---":
                    resep = self.semua_resep_dict.get(nama_resep)
                    if resep:
                        for bahan in resep.bahan:
                            kunci = (bahan.nama.strip().capitalize(), bahan.satuan.strip().lower())
                            daftar_belanja[kunci] += bahan.jumlah

        return dict(sorted(daftar_belanja.items()))

Overwriting perencana.py


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import random
from model import Bahan, Resep
from manajer_data import ManajerData
from perencana import PerencanaMenu

st.set_page_config(page_title="Asisten Dapur Cerdas", layout="wide", page_icon="🍳")

css = """
/* (CSS sama seperti sebelumnya, untuk menjaga tampilan) */
:root { --primary-color: #FF6B6B; --background-color: #1a1a1a; --secondary-background-color: #2b2b2b; --text-color: #EAEAEA; --font: "Segoe UI", sans-serif; }
.stButton>button { border-color: var(--primary-color); color: var(--primary-color); border-radius: 20px; transition: all 0.2s ease-in-out; }
.stButton>button:hover { border-color: #FFFFFF; color: #FFFFFF; background-color: var(--primary-color); }
.stButton>button:focus { border-color: #FFFFFF; color: #FFFFFF; background-color: var(--primary-color); box-shadow: none; }
"""
st.markdown(f"<style>{css}</style>", unsafe_allow_html=True)

if 'manajer' not in st.session_state:
    st.session_state.manajer = ManajerData()
if 'daftar_resep' not in st.session_state:
    st.session_state.daftar_resep = st.session_state.manajer.muat_resep()
if 'rencana_menu_df' not in st.session_state:
    hari = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu", "Minggu"]
    waktu_makan = ["Pagi", "Siang", "Malam"]
    df = pd.DataFrame(columns=hari, index=waktu_makan); df[:] = "---"
    st.session_state.rencana_menu_df = df

def reload_recipes():
    st.session_state.daftar_resep = st.session_state.manajer.muat_resep()

st.title("🍳 Asisten Dapur Cerdas ")
st.caption("Perencanaan Kuliner Harian di Ujung Jari Anda.")

tab_beranda, tab_resep, tab_rencana, tab_belanja = st.tabs(["**🏠 Beranda**", "**📖 Kelola Resep**", "**📅 Rencanakan Menu**", "**🛒 Daftar Belanja**"])

with tab_beranda:
    st.header("Selamat Datang di Asisten Dapur Anda!")
    col1, col2 = st.columns(2)
    total_resep = len(st.session_state.daftar_resep)
    col1.metric("Total Resep Tersimpan", f"{total_resep} 📖")
    menu_direncanakan = (st.session_state.rencana_menu_df != "---").sum().sum()
    col2.metric("Menu Direncanakan Minggu Ini", f"{menu_direncanakan} 🍽️")
    st.divider()
    st.subheader("✨ Resep Pilihan Hari Ini")
    if total_resep > 0:
        resep_pilihan = random.choice(st.session_state.daftar_resep)
        with st.container(border=True): st.markdown(f"#### {resep_pilihan.nama}")
    else:
        st.info("Tambahkan resep pertama Anda di tab 'Kelola Resep'.")


with tab_resep:
    with st.expander("➕ Tambah atau Edit Resep"):
        if 'edit_mode' not in st.session_state: st.session_state.edit_mode = False
        resep_sebelumnya = st.session_state.get('resep_to_edit', None)
        with st.form(key="form_resep"):
            st.subheader("Edit Resep" if resep_sebelumnya else "Tambah Resep Baru")
            nama_resep = st.text_input("Nama Resep", value=resep_sebelumnya.nama if resep_sebelumnya else "")
            bahan_awal = "\n".join([f"{b.nama},{b.jumlah},{b.satuan}" for b in resep_sebelumnya.bahan]) if resep_sebelumnya else ""
            bahan_text = st.text_area("Bahan-bahan (format: Nama,Jumlah,Satuan)", value=bahan_awal, height=150)
            instruksi = st.text_area("Instruksi", value=resep_sebelumnya.instruksi if resep_sebelumnya else "")

            submit_col, cancel_col, _ = st.columns([1,1,4])
            if submit_col.form_submit_button("Simpan"):
                if not nama_resep or not bahan_text:
                    st.error("Nama resep dan bahan tidak boleh kosong!")
                else:
                    try:
                        bahan_list = []
                        for line in bahan_text.strip().split('\n'):
                            if line.strip():
                                parts = line.split(',')
                                if len(parts) == 3:
                                    nama = parts[0].strip()
                                    jumlah = float(parts[1].strip())
                                    satuan = parts[2].strip()
                                    bahan_list.append(Bahan(nama=nama, jumlah=jumlah, satuan=satuan))
                                else:
                                    raise ValueError(f"Format baris bahan salah: '{line}'")

                        resep_obj = Resep(nama=nama_resep, bahan=bahan_list, instruksi=instruksi)

                        if resep_sebelumnya: st.session_state.manajer.update_resep(resep_sebelumnya.nama, resep_obj)
                        else: st.session_state.manajer.tambah_resep(resep_obj)

                        st.session_state.edit_mode = False; st.session_state.resep_to_edit = None
                        reload_recipes()
                        st.success("Resep berhasil disimpan!")
                        st.rerun()

                    except Exception as e:
                        st.error(f"Gagal menyimpan: {e}")

            if cancel_col.form_submit_button("Batal", type="secondary"):
                st.session_state.edit_mode = False; st.session_state.resep_to_edit = None; st.rerun()

    st.header("Koleksi Resep Anda")
    if not st.session_state.daftar_resep: st.info("Belum ada resep.")
    else:
        for resep in st.session_state.daftar_resep:
            with st.container(border=True):
                col1, col2 = st.columns([3, 1])
                with col1:
                    st.subheader(resep.nama)
                    st.caption(f"Jumlah Bahan: {len(resep.bahan)}")
                with col2:
                    action_col1, action_col2 = st.columns(2)
                    if action_col1.button("👁️", key=f"view_{resep.nama}", help="Lihat Detail"):
                        st.session_state.resep_to_edit = resep
                    if action_col2.button("❌", key=f"del_{resep.nama}", help="Hapus Resep"):
                        st.session_state.manajer.hapus_resep(resep.nama); reload_recipes(); st.rerun()

                if st.session_state.get('resep_to_edit') and st.session_state.resep_to_edit.nama == resep.nama:
                    st.divider()
                    st.markdown("**Bahan-bahan:**")
                    for bahan in resep.bahan: st.markdown(f"- {bahan.jumlah} {bahan.satuan} **{bahan.nama}**")
                    st.markdown("**Instruksi:**")
                    st.write(resep.instruksi if resep.instruksi else "Tidak ada instruksi.")
                    if st.button("Tutup Detail", key=f"close_{resep.nama}"):
                        st.session_state.resep_to_edit = None; st.rerun()


with tab_rencana:
    st.header("Rencana Menu Mingguan")
    st.info("Klik pada sel untuk memilih resep dari daftar Anda.")
    pilihan_resep = ["---"] + [r.nama for r in st.session_state.daftar_resep]
    edited_df = st.data_editor(st.session_state.rencana_menu_df, column_config={col: st.column_config.SelectboxColumn(options=pilihan_resep, required=True) for col in st.session_state.rencana_menu_df.columns}, use_container_width=True, height=145)
    st.session_state.rencana_menu_df = edited_df

with tab_belanja:
    st.header("Daftar Belanja Otomatis")
    rencana_dict = st.session_state.rencana_menu_df.T.to_dict('list')
    perencana = PerencanaMenu(rencana_dict, st.session_state.daftar_resep)
    daftar_belanja_dict = perencana.buat_daftar_belanja()
    if not daftar_belanja_dict: st.info("Rencana menu Anda kosong.")
    else:
        df_belanja = pd.DataFrame([{"Bahan": nama, "Jumlah": jumlah, "Satuan": satuan} for (nama, satuan), jumlah in daftar_belanja_dict.items()])
        col1, col2 = st.columns(2)
        with col1: st.dataframe(df_belanja, use_container_width=True, hide_index=True)
        with col2: st.bar_chart(df_belanja.set_index('Bahan')['Jumlah'])
        csv = df_belanja.to_csv(index=False).encode('utf-8')
        st.download_button("📥 Unduh Daftar Belanja (.csv)", csv, 'daftar_belanja.csv', 'text/csv')

Overwriting app.py


In [ ]:
from pyngrok import ngrok
import os

AUTHTOKEN = "2zDE01de6QGaCSBQvEm0fe6fuh3_7aqUaZcE1pgx9U41cieH"

os.environ["NGROK_AUTHTOKEN"] = AUTHTOKEN
ngrok.set_auth_token(AUTHTOKEN)

!killall ngrok
!nohup streamlit run app.py &

try:
    public_url = ngrok.connect(8501)
    print("\n=======================================================================================")
    print("🚀 APLIKASI WEB ANDA (v4.0) SUDAH SIAP!")
    print(f"🔗 Klik link berikut untuk membuka: {public_url}")
    print("=======================================================================================")
except Exception as e:
    print(f"Error saat menjalankan ngrok: {e}")

nohup: appending output to 'nohup.out'

🚀 APLIKASI WEB ANDA (v4.0) SUDAH SIAP!
🔗 Klik link berikut untuk membuka: NgrokTunnel: "https://69d1-35-243-234-90.ngrok-free.app" -> "http://localhost:8501"
